# Classification Experiment: Friends
---
This Notebook, includes a series of experiments, on using a node's Friends for classification.

Libraries:

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
import json
import tweepy
import time

from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import FeatureUnion

from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

import joblib
import time 

import seaborn as sns

Twitter API Authentication:

In [5]:
twitter_credentials = []
with open('../../../../twitter_credentials.json', 'r') as f:
    twitter_credentials = json.load(f)

auth = tweepy.OAuthHandler(twitter_credentials['consumer_key'], twitter_credentials['consumer_secret'])
auth.set_access_token(twitter_credentials['access_token_key'],twitter_credentials['access_token_secret'])
API = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, timeout=60*5)

In [6]:
# Function For Text Normalization
def clean_text(data):
    urls = r'http\S+'
    non_unicode_char = r'\W'
    numbers = r'[0-9_]'
    fix_whitespace = r'\s+'
    single_whitespace = ' '
    
    data = (data.replace([urls], single_whitespace, regex=True)
                    .replace([non_unicode_char, numbers], single_whitespace, regex=True)
                    .replace(fix_whitespace, single_whitespace, regex=True))
    data = data.apply(lambda s: s.lower() if type(s) == str else s)
    return data

nlp_el = spacy.load('el_core_news_md')
nlp_en = spacy.load('en_core_web_sm')
STOPWORDS = set(list(spacy.lang.en.STOP_WORDS) + list(spacy.lang.el.STOP_WORDS))

def remove_stopwords(row):
    row = [str(token) for token in nlp_el(row)]
    return [w for w in row if w not in STOPWORDS]

def tokenize_lemmatize(row):
    return [str(token.lemma_) for token in nlp_el(row)]

# Dataset
---

In [7]:
# Read Training Set
training_set = pd.read_csv('../../../../datasets/Greek Politicians/classification/parliament_members_training_set.csv')
training_set = training_set.replace(np.nan, '')

In [11]:
training_set.tail()

,screen_name,name,description,statuses_count,recent_100_tweets,friends_count,recent_100_friends_nd,followers_count,default_profile_image,favourites_count,parliament_member,friends_politician_count_1000,followers_politician_count_1000,mentions_politician_count
295,manoskonsolas,Manos Konsolas,Βουλευτής Δωδεκανήσου - Καθηγητής Πανεπιστημίο...,5467,Λανθασμένη επιλογή η αναστολή λειτουργίας των...,1534,Kostas Tsiaras Υπουργός Δικαιοσύνης & Βουλευτ...,3757,False,151,1,101,10.0,1
296,vnaypsilantis,Vassilis Ypsilantis,Α’ Κοσμήτορας της Βουλής των Ελλήνων | Βουλευτ...,1240,RT @grigoris_d: Corriere della Sera: Το Καστε...,582,Ιωάννης Μπούγας ΙΩΑΝΝΗΣ Δ ΜΠΟΥΓΑΣ \nΔικηγόρος...,956,False,676,1,122,11.0,16
297,FofiGennimata,Fofi Gennimata,Πρόεδρος του Κινήματος Αλλαγής | Πρόεδρος του ...,9248,"Η Μελίνα των οραμάτων, των μαρμάρων, του πολι...",2380,Τομέας Απόδημου Ελληνισμού - Κίνημα Αλλαγής ...,45893,False,421,1,91,2.0,0
298,ChitasKostas,Chitas Constantinos,Βουλευτής Β´ Θεσσαλονίκης Ελληνική Λύση ...,5632,Σήμερα στις 12:00 στον «αέρα» του @FOCUSFM103...,599,Ελληνες Ενωμένοι επιτέλους Μακεδόνισσα λατρεύ...,997,False,12328,1,51,2.0,66
299,KastanidisHaris,Kastanidis Haris,Όλη η ζωή είναι μια ξένη χώρα.\r\nJack Kerouac,4117,Οι μικρές και μεσαίες επιχειρήσεις αδυνατούν ...,700,"Andreas Spiropoulos Μηχανικός, Μέλος Πολιτικο...",3841,False,1281,1,74,6.0,3


# Only Counts

In [6]:
X = training_set
y = training_set.parliament_member

In [7]:
# Function to select the data
def get_data_(df):
    data = df['friends_politician_count_1000'].to_numpy()
    return data.reshape(-1,1)


get_data = FunctionTransformer(get_data_)


print('------------------ Support Vector Machine -------------------\n')

# The pipeline
pipeline = Pipeline([
    ('selector', get_data),
    ('svm', svm.SVC())
])

# Paramters for optimization
parameters = {'svm__C' : [0.1,0.5,1,5,10],
              'svm__kernel':['linear', 'poly', 'rbf', 'sigmoid']}

grid = GridSearchCV(pipeline, parameters, n_jobs = 4)
grid.fit(X, y)
    
print(f' Best Params: {grid.best_params_}.\n Score: {grid.best_score_}')


print('\n\n------------------ kNN -------------------\n')

# The pipeline
pipeline = Pipeline([
    ('selector', get_data),
    ('knn', KNeighborsClassifier())
])

# Paramters for optimization
parameters = {'knn__n_neighbors': [i for i in range(20)],
              'knn__weights': ['uniform', 'distance']}

grid = GridSearchCV(pipeline, parameters, n_jobs = 4)
grid.fit(X, y)
    
print(f' Best Params: {grid.best_params_}.\n Score: {grid.best_score_}')

print('\n\n------------------ Logistic Regression -------------------\n')

# The pipeline
pipeline = Pipeline([
    ('selector', get_data),
    ('lr', LogisticRegression(max_iter=1000))
])

# Paramters for optimization
parameters = {'lr__penalty': ['l1', 'l2', 'elasticnet'],
              'lr__C': [0.1, 0.5, 1, 5, 10]}

grid = GridSearchCV(pipeline, parameters, n_jobs = 4)
grid.fit(X, y)
    
print(f' Best Params: {grid.best_params_}.\n Score: {grid.best_score_}')

------------------ Support Vector Machine -------------------

 Best Params: {'svm__C': 0.5, 'svm__kernel': 'rbf'}.
 Score: 0.8666666666666668


------------------ kNN -------------------



/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.79333333 0.79333333 0.76333333 0.80666667
 0.86333333 0.84       0.86       0.85       0.87333333 0.84666667
 0.86666667 0.85       0.87666667 0.85       0.87       0.85
 0.87333333 0.85333333 0.87       0.85333333 0.87       0.85333333
 0.87       0.85333333 0.87       0.85333333 0.86666667 0.85333333
 0.87       0.85333333 0.86666667 0.85333333 0.87       0.85333333
 0.86666667 0.85333333 0.87       0.85333333]
  warnings.warn(


 Best Params: {'knn__n_neighbors': 7, 'knn__weights': 'uniform'}.
 Score: 0.8766666666666667


------------------ Logistic Regression -------------------

 Best Params: {'lr__C': 0.1, 'lr__penalty': 'l2'}.
 Score: 0.8433333333333334


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan 0.84333333        nan        nan 0.84333333        nan
        nan 0.84333333        nan        nan 0.84333333        nan
        nan 0.84333333        nan]
  warnings.warn(


### Export Model

In [9]:
X = training_set
y = training_set.parliament_member

# Function to select the data
def get_data_parl_fr(df):
    data = df['friends_politician_count_1000'].to_numpy()
    return data.reshape(-1,1)


get_data = FunctionTransformer(get_data_parl_fr)

# The pipeline
pipeline = Pipeline([
    ('selector', get_data),
    ('knn', KNeighborsClassifier(n_neighbors=7, weights='uniform'))
])

pipeline.fit(X,y)

Pipeline(steps=[('selector',
                 FunctionTransformer(func=<function get_data_parl_fr at 0x7fab5967fb80>)),
                ('knn', KNeighborsClassifier(n_neighbors=7))])

In [10]:
filename = 'classifier_parl_fr.sav'
joblib.dump(pipeline, filename)

['classifier_parl_fr.sav']

# Name Description Tweets and Counts

## Without NLP

In [8]:
train = training_set.copy()
train['textdata'] = clean_text(train['name'] + ' ' + train['description'] + ' ' + train['recent_100_tweets'])
X = train
y = train.parliament_member

In [9]:
def get_text_data_(df):
    
    return df.textdata

get_text_data = FunctionTransformer(get_text_data_)

def get_numeric_data_(df):
    data = df['friends_politician_count_1000'].to_numpy()
    return data.reshape(-1,1)

get_numeric_data = FunctionTransformer(get_numeric_data_)




print('------------------ Support Vector Machine -------------------\n')

pipeline = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector_num', get_numeric_data)
            ])),
             ('text_features', Pipeline([
                ('selector_text', get_text_data),
                ('vectorizer', TfidfVectorizer()),
            ]))
         ])),
     ('svm', svm.SVC())
])


# Paramters for optimization
parameters = {'features__text_features__vectorizer__max_df': [0.5, 0.75, 1],
              'features__text_features__vectorizer__min_df': [1, 5, 10],
              'features__text_features__vectorizer__max_features': [1000, 2000, None],
              'svm__C' : [0.1,0.5,1,5,10],
              'svm__kernel':['linear', 'poly', 'rbf', 'sigmoid']
                  }

grid = GridSearchCV(pipeline, parameters, n_jobs = 4, verbose=1)
grid.fit(X, y)
    
print(f' Best Params: {grid.best_params_}.\n Score: {grid.best_score_}')


print('\n\n------------------ kNN -------------------\n')

pipeline = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector_num', get_numeric_data)
            ])),
             ('text_features', Pipeline([
                ('selector_text', get_text_data),
                ('vectorizer', TfidfVectorizer()),
            ]))
         ])),
     ('knn', KNeighborsClassifier())
])


# Paramters for optimization
parameters = {'features__text_features__vectorizer__max_df': [0.5, 0.75, 1],
              'features__text_features__vectorizer__min_df': [1, 5, 10],
              'features__text_features__vectorizer__max_features': [1000, 2000, None],
              'knn__n_neighbors': [1,2,3,4,5,6,7,8,9,10],
              'knn__weights': ['uniform', 'distance']
                  }

grid = GridSearchCV(pipeline, parameters, n_jobs = 4, verbose=1)
grid.fit(X, y)
    
print(f' Best Params: {grid.best_params_}.\n Score: {grid.best_score_}')


print('\n\n------------------ Logistic Regression -------------------\n')


pipeline = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector_num', get_numeric_data)
            ])),
             ('text_features', Pipeline([
                ('selector_text', get_text_data),
                ('vectorizer', TfidfVectorizer()),
            ]))
         ])),
     ('lr', LogisticRegression(max_iter=1000))
])


# Paramters for optimization
parameters = {'features__text_features__vectorizer__max_df': [0.5, 0.75, 1],
              'features__text_features__vectorizer__min_df': [1, 5, 10],
              'features__text_features__vectorizer__max_features': [1000, 2000, None],
              'lr__penalty': ['l1', 'l2', 'elasticnet', 'none'],
              'lr__C': [0.1, 0.5, 1, 5, 10]
                  }

grid = GridSearchCV(pipeline, parameters, n_jobs = 4, verbose=1)
grid.fit(X, y)
    
print(f' Best Params: {grid.best_params_}.\n Score: {grid.best_score_}')

del train

------------------ Support Vector Machine -------------------

Fitting 5 folds for each of 540 candidates, totalling 2700 fits


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.87       0.75       0.84666667 0.85       0.88666667 0.76
 0.86       0.83666667 0.89333333 0.76       0.87       0.83333333
 0.89666667 0.77       0.86666667 0.83333333 0.89666667 0.77
 0.87       0.83       0.87       0.75       0.84666667 0.85
 0.89333333 0.76       0.86       0.83666667 0.90333333 0.76
 0.87       0.83333333 0.89333333 0.77       0.86666667 0.83333333
 0.89       0.77       0.87       0.83       0.87       0.75
 0.84666667 0.85       0.88666667 0.76       0.86       0.83666667
 0.89333333 0.76       0.87       0.83333333 0.88       0.77
 0.86666667 0.83333333 0.87666667 0.77       0.87       0.83
 0.86666667 0.75       0.84666667 0.85       0.88666667 0.76
 0.86       0.83666667 0.89666667 0.76       0.87       0.83333333
 0.89       0.77       0.86666667 0.83333333 0.89       0.77
 0.87

 Best Params: {'features__text_features__vectorizer__max_df': 0.75, 'features__text_features__vectorizer__max_features': 1000, 'features__text_features__vectorizer__min_df': 1, 'svm__C': 1, 'svm__kernel': 'linear'}.
 Score: 0.9133333333333333


------------------ kNN -------------------

Fitting 5 folds for each of 540 candidates, totalling 2700 fits


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.83       0.83       0.79666667 0.83       0.87       0.86666667
 0.86333333 0.86666667 0.87333333 0.87666667 0.87       0.87333333
 0.87666667 0.87666667 0.87       0.87666667 0.87333333 0.88
 0.87       0.87666667 0.83333333 0.83333333 0.79333333 0.83333333
 0.86666667 0.86333333 0.86333333 0.86666667 0.87333333 0.87666667
 0.87       0.87333333 0.87666667 0.87666667 0.87       0.87666667
 0.87333333 0.88       0.87       0.87666667 0.83333333 0.83333333
 0.79333333 0.83333333 0.87       0.86666667 0.86333333 0.86666667
 0.87666667 0.87666667 0.87       0.87333333 0.87666667 0.87666667
 0.87       0.87666667 0.87333333 0.88       0.87       0.88
 0.83       0.83       0.79333333 0.83       0.87333333 0.87
 0.86333333 0.86666667 0.87666667 0.87666667 0.87       0.87333333
 0.87666667 0.87666667 0.87       0.

 Best Params: {'features__text_features__vectorizer__max_df': 0.5, 'features__text_features__vectorizer__max_features': 1000, 'features__text_features__vectorizer__min_df': 1, 'knn__n_neighbors': 9, 'knn__weights': 'distance'}.
 Score: 0.8800000000000001


------------------ Logistic Regression -------------------

Fitting 5 folds for each of 540 candidates, totalling 2700 fits


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan 0.85666667        nan 0.9               nan 0.87666667
        nan 0.9               nan 0.87666667        nan 0.9
        nan 0.91              nan 0.9               nan 0.91333333
        nan 0.9               nan 0.85666667        nan 0.90333333
        nan 0.87666667        nan 0.90333333        nan 0.87666667
        nan 0.90333333        nan 0.90666667        nan 0.90333333
        nan 0.90333333        nan 0.90333333        nan 0.85666667
        nan 0.91              nan 0.87666667        nan 0.91
        nan 0.87666667        nan 0.91              nan 0.90333333
        nan 0.91              nan 0.90666667        nan 0.91
        nan 0.85333333        nan 0.91333333        nan 0.87333333
        nan 0.91333333        nan 0.87666667        nan 0.91333333
        nan 0.90666667        nan 0.9

 Best Params: {'features__text_features__vectorizer__max_df': 0.5, 'features__text_features__vectorizer__max_features': 1000, 'features__text_features__vectorizer__min_df': 1, 'lr__C': 10, 'lr__penalty': 'l2'}.
 Score: 0.9133333333333334


## Stop Word Removal

In [10]:
train = training_set.copy()
train['textdata'] = clean_text(train['name'] + ' ' + train['description'] + ' ' + train['recent_100_tweets'])
train['textdata'] = train['textdata'].apply(lambda row: remove_stopwords(row))
train['textdata'] = train['textdata'].apply(lambda row: ' '.join(row))

X = train
y = train.parliament_member

In [11]:
def get_text_data_(df):
    
    return df.textdata

get_text_data = FunctionTransformer(get_text_data_)


def get_numeric_data_(df):
    data = df['friends_politician_count_1000'].to_numpy()
    return data.reshape(-1,1)

get_numeric_data = FunctionTransformer(get_numeric_data_)




print('------------------ Support Vector Machine -------------------\n')

pipeline = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector_num', get_numeric_data)
            ])),
             ('text_features', Pipeline([
                ('selector_text', get_text_data),
                ('vectorizer', TfidfVectorizer()),
            ]))
         ])),
     ('svm', svm.SVC())
])


# Paramters for optimization
parameters = {'features__text_features__vectorizer__max_df': [0.5, 0.75, 1],
              'features__text_features__vectorizer__min_df': [1, 5, 10],
              'features__text_features__vectorizer__max_features': [1000, 2000, None],
              'svm__C' : [0.1,0.5,1,5,10],
              'svm__kernel':['linear', 'poly', 'rbf', 'sigmoid']
                  }

grid = GridSearchCV(pipeline, parameters, n_jobs = 4, verbose=1)
grid.fit(X, y)
    
print(f' Best Params: {grid.best_params_}.\n Score: {grid.best_score_}')


print('\n\n------------------ kNN -------------------\n')

pipeline = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector_num', get_numeric_data)
            ])),
             ('text_features', Pipeline([
                ('selector_text', get_text_data),
                ('vectorizer', TfidfVectorizer()),
            ]))
         ])),
     ('knn', KNeighborsClassifier())
])


# Paramters for optimization
parameters = {'features__text_features__vectorizer__max_df': [0.5, 0.75, 1],
              'features__text_features__vectorizer__min_df': [1, 5, 10],
              'features__text_features__vectorizer__max_features': [1000, 2000, None],
              'knn__n_neighbors': [1,2,3,4,5,6,7,8,9,10],
              'knn__weights': ['uniform', 'distance']
                  }

grid = GridSearchCV(pipeline, parameters, n_jobs = 4, verbose=1)
grid.fit(X, y)
    
print(f' Best Params: {grid.best_params_}.\n Score: {grid.best_score_}')


print('\n\n------------------ Logistic Regression -------------------\n')


pipeline = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector_num', get_numeric_data)
            ])),
             ('text_features', Pipeline([
                ('selector_text', get_text_data),
                ('vectorizer', TfidfVectorizer()),
            ]))
         ])),
     ('lr', LogisticRegression(max_iter=1000))
])


# Paramters for optimization
parameters = {'features__text_features__vectorizer__max_df': [0.5, 0.75, 1],
              'features__text_features__vectorizer__min_df': [1, 5, 10],
              'features__text_features__vectorizer__max_features': [1000, 2000, None],
              'lr__penalty': ['l1', 'l2', 'elasticnet', 'none'],
              'lr__C': [0.1, 0.5, 1, 5, 10]
                  }

grid = GridSearchCV(pipeline, parameters, n_jobs = 4, verbose=1)
grid.fit(X, y)
    
print(f' Best Params: {grid.best_params_}.\n Score: {grid.best_score_}')

del train

------------------ Support Vector Machine -------------------

Fitting 5 folds for each of 540 candidates, totalling 2700 fits


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.86666667 0.75       0.84666667 0.85       0.88333333 0.76
 0.86       0.83666667 0.90333333 0.76       0.87       0.83333333
 0.91666667 0.77       0.86666667 0.83333333 0.90666667 0.77
 0.87       0.83       0.87       0.75       0.84666667 0.85
 0.89       0.76       0.86       0.83666667 0.90666667 0.76
 0.87       0.83333333 0.9        0.77       0.86666667 0.83333333
 0.9        0.77       0.87       0.83       0.87       0.75
 0.84666667 0.85       0.88333333 0.76       0.86       0.83666667
 0.90333333 0.76       0.87       0.83333333 0.9        0.77
 0.86666667 0.83333333 0.90333333 0.77       0.87       0.83
 0.86333333 0.75       0.84666667 0.85       0.88666667 0.76
 0.86       0.83666667 0.89666667 0.76       0.87       0.83333333
 0.90333333 0.77       0.86666667 0.83333333 0.9        0.77
 0.87

 Best Params: {'features__text_features__vectorizer__max_df': 0.5, 'features__text_features__vectorizer__max_features': 1000, 'features__text_features__vectorizer__min_df': 1, 'svm__C': 5, 'svm__kernel': 'linear'}.
 Score: 0.9166666666666666


------------------ kNN -------------------

Fitting 5 folds for each of 540 candidates, totalling 2700 fits


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.83333333 0.83333333 0.79666667 0.83333333 0.87333333 0.87
 0.86333333 0.86666667 0.87333333 0.87666667 0.87       0.87333333
 0.87666667 0.87666667 0.87       0.87666667 0.87333333 0.88
 0.87       0.87666667 0.83       0.83       0.8        0.83
 0.87       0.86666667 0.86333333 0.86666667 0.87333333 0.87666667
 0.87       0.87333333 0.87666667 0.87666667 0.87       0.87666667
 0.87333333 0.88       0.87       0.87666667 0.83       0.83
 0.8        0.83       0.87       0.86666667 0.86666667 0.86666667
 0.87666667 0.87666667 0.87       0.87333333 0.87666667 0.87666667
 0.87       0.87666667 0.87333333 0.88       0.87       0.88
 0.83333333 0.83333333 0.79       0.83333333 0.87666667 0.87333333
 0.86333333 0.86666667 0.87666667 0.87666667 0.87       0.87333333
 0.87666667 0.87666667 0.87       0.87666667 0.8

 Best Params: {'features__text_features__vectorizer__max_df': 0.5, 'features__text_features__vectorizer__max_features': 1000, 'features__text_features__vectorizer__min_df': 1, 'knn__n_neighbors': 9, 'knn__weights': 'distance'}.
 Score: 0.8800000000000001


------------------ Logistic Regression -------------------

Fitting 5 folds for each of 540 candidates, totalling 2700 fits


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan 0.85666667        nan 0.90333333        nan 0.87333333
        nan 0.90333333        nan 0.88333333        nan 0.90333333
        nan 0.90666667        nan 0.90333333        nan 0.91333333
        nan 0.90333333        nan 0.85666667        nan 0.91333333
        nan 0.87333333        nan 0.91333333        nan 0.88333333
        nan 0.91333333        nan 0.90666667        nan 0.91333333
        nan 0.91333333        nan 0.91333333        nan 0.85666667
        nan 0.90666667        nan 0.87666667        nan 0.90666667
        nan 0.88333333        nan 0.90666667        nan 0.90333333
        nan 0.90666667        nan 0.91              nan 0.90666667
        nan 0.85666667        nan 0.90666667        nan 0.87333333
        nan 0.90666667        nan 0.88333333        nan 0.90666667
        nan 0.9066

 Best Params: {'features__text_features__vectorizer__max_df': 0.5, 'features__text_features__vectorizer__max_features': 2000, 'features__text_features__vectorizer__min_df': 5, 'lr__C': 0.1, 'lr__penalty': 'none'}.
 Score: 0.9166666666666667


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


## Lemmatization and Stop Word Removal

In [12]:
train = training_set.copy()
train['textdata'] = clean_text(train['name'] + ' ' + train['description'] + ' ' + train['recent_100_tweets'])
train['textdata'] = train['textdata'].apply(lambda row: tokenize_lemmatize(row))
train['textdata'] = train['textdata'].apply(lambda row: ' '.join(row))
train['textdata'] = train['textdata'].apply(lambda row: remove_stopwords(row))
train['textdata'] = train['textdata'].apply(lambda row: ' '.join(row))
X = train
y = train.parliament_member

In [13]:
def get_text_data_(df):
    return df.textdata

get_text_data = FunctionTransformer(get_text_data_)


def get_numeric_data_(df):
    data = df['friends_politician_count_1000'].to_numpy()
    return data.reshape(-1,1)

get_numeric_data = FunctionTransformer(get_numeric_data_)




print('------------------ Support Vector Machine -------------------\n')

pipeline = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector_num', get_numeric_data)
            ])),
             ('text_features', Pipeline([
                ('selector_text', get_text_data),
                ('vectorizer', TfidfVectorizer()),
            ]))
         ])),
     ('svm', svm.SVC())
])


# Paramters for optimization
parameters = {'features__text_features__vectorizer__max_df': [0.5, 0.75, 1],
              'features__text_features__vectorizer__min_df': [1, 5, 10],
              'features__text_features__vectorizer__max_features': [1000, 2000, None],
              'svm__C' : [0.1,0.5,1,5,10],
              'svm__kernel':['linear', 'poly', 'rbf', 'sigmoid']
                  }

grid = GridSearchCV(pipeline, parameters, n_jobs = 4, verbose=1)
grid.fit(X, y)
    
print(f' Best Params: {grid.best_params_}.\n Score: {grid.best_score_}')


print('\n\n------------------ kNN -------------------\n')

pipeline = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector_num', get_numeric_data)
            ])),
             ('text_features', Pipeline([
                ('selector_text', get_text_data),
                ('vectorizer', TfidfVectorizer()),
            ]))
         ])),
     ('knn', KNeighborsClassifier())
])


# Paramters for optimization
parameters = {'features__text_features__vectorizer__max_df': [0.5, 0.75, 1],
              'features__text_features__vectorizer__min_df': [1, 5, 10],
              'features__text_features__vectorizer__max_features': [1000, 2000, None],
              'knn__n_neighbors': [1,2,3,4,5,6,7,8,9,10],
              'knn__weights': ['uniform', 'distance']
                  }

grid = GridSearchCV(pipeline, parameters, n_jobs = 4, verbose=1)
grid.fit(X, y)
    
print(f' Best Params: {grid.best_params_}.\n Score: {grid.best_score_}')


print('\n\n------------------ Logistic Regression -------------------\n')


pipeline = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector_num', get_numeric_data)
            ])),
             ('text_features', Pipeline([
                ('selector_text', get_text_data),
                ('vectorizer', TfidfVectorizer()),
            ]))
         ])),
     ('lr', LogisticRegression(max_iter=1000))
])


# Paramters for optimization
parameters = {'features__text_features__vectorizer__max_df': [0.5, 0.75, 1],
              'features__text_features__vectorizer__min_df': [1, 5, 10],
              'features__text_features__vectorizer__max_features': [1000, 2000, None],
              'lr__penalty': ['l1', 'l2', 'elasticnet', 'none'],
              'lr__C': [0.1, 0.5, 1, 5, 10]
                  }

grid = GridSearchCV(pipeline, parameters, n_jobs = 4, verbose=1)
grid.fit(X, y)
    
print(f' Best Params: {grid.best_params_}.\n Score: {grid.best_score_}')

del train

------------------ Support Vector Machine -------------------

Fitting 5 folds for each of 540 candidates, totalling 2700 fits


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.87       0.75       0.84666667 0.85       0.88666667 0.76
 0.86       0.83666667 0.89333333 0.76       0.87       0.83333333
 0.9        0.77       0.86666667 0.83333333 0.9        0.77
 0.87       0.83       0.87       0.75       0.84666667 0.85
 0.89666667 0.76       0.86       0.83666667 0.9        0.76
 0.87       0.83333333 0.9        0.77       0.86666667 0.83333333
 0.9        0.77       0.87       0.83       0.87333333 0.75
 0.84666667 0.85       0.88       0.76       0.86       0.83666667
 0.89       0.76       0.87       0.83333333 0.9        0.77
 0.86666667 0.83333333 0.89666667 0.77       0.87       0.83
 0.86333333 0.75       0.84666667 0.85       0.88333333 0.76
 0.86       0.83666667 0.89333333 0.76       0.87       0.83333333
 0.88666667 0.77       0.86666667 0.83333333 0.88666667 0.77
 0.87

 Best Params: {'features__text_features__vectorizer__max_df': 0.75, 'features__text_features__vectorizer__max_features': 1000, 'features__text_features__vectorizer__min_df': 1, 'svm__C': 1, 'svm__kernel': 'linear'}.
 Score: 0.9066666666666666


------------------ kNN -------------------

Fitting 5 folds for each of 540 candidates, totalling 2700 fits


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.82666667 0.82666667 0.79666667 0.82666667 0.87333333 0.87
 0.86333333 0.87       0.87333333 0.87666667 0.87       0.87333333
 0.87666667 0.87666667 0.87       0.87666667 0.87333333 0.88
 0.87       0.88       0.83       0.83       0.8        0.83
 0.87       0.86666667 0.86333333 0.87       0.87333333 0.87666667
 0.87       0.87333333 0.87666667 0.87666667 0.87       0.87666667
 0.87333333 0.88       0.87       0.88       0.83       0.83
 0.79666667 0.83       0.87       0.86666667 0.86333333 0.87
 0.87666667 0.87666667 0.87       0.87333333 0.87666667 0.87666667
 0.87       0.87666667 0.87333333 0.88       0.87       0.88
 0.82333333 0.82333333 0.79666667 0.82333333 0.87333333 0.87
 0.86333333 0.86666667 0.87666667 0.87666667 0.87       0.87333333
 0.87666667 0.87666667 0.87       0.87666667 0.87333333 0.88

 Best Params: {'features__text_features__vectorizer__max_df': 0.5, 'features__text_features__vectorizer__max_features': 1000, 'features__text_features__vectorizer__min_df': 1, 'knn__n_neighbors': 9, 'knn__weights': 'distance'}.
 Score: 0.8800000000000001


------------------ Logistic Regression -------------------

Fitting 5 folds for each of 540 candidates, totalling 2700 fits


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan 0.85333333        nan 0.89666667        nan 0.87333333
        nan 0.89666667        nan 0.88              nan 0.89666667
        nan 0.90333333        nan 0.89666667        nan 0.9
        nan 0.89666667        nan 0.85333333        nan 0.88666667
        nan 0.87333333        nan 0.88666667        nan 0.88
        nan 0.88666667        nan 0.9               nan 0.88666667
        nan 0.9               nan 0.88666667        nan 0.85333333
        nan 0.89666667        nan 0.87666667        nan 0.89666667
        nan 0.87666667        nan 0.89666667        nan 0.89333333
        nan 0.89666667        nan 0.89666667        nan 0.89666667
        nan 0.85333333        nan 0.88333333        nan 0.87333333
        nan 0.88333333        nan 0.87666667        nan 0.88333333
        nan 0.89333333        n

 Best Params: {'features__text_features__vectorizer__max_df': 0.5, 'features__text_features__vectorizer__max_features': None, 'features__text_features__vectorizer__min_df': 10, 'lr__C': 0.1, 'lr__penalty': 'none'}.
 Score: 0.9133333333333333


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


### Export Model

In [18]:
X = training_set
y = training_set.parliament_member

In [20]:
def get_text_data_(df):
    df = df.copy()
    df['textdata'] = clean_text(df['name']+ ' ' + df['description'] + ' ' + df['recent_100_tweets'])
    df['textdata'] = df['textdata'].apply(lambda row: remove_stopwords(row))
    df['textdata'] = df['textdata'].apply(lambda row: ' '.join(row))
    return df.textdata

get_text_data = FunctionTransformer(get_text_data_)


def get_numeric_data_(df):
    data = df['friends_politician_count_1000'].to_numpy()
    return data.reshape(-1,1)

get_numeric_data = FunctionTransformer(get_numeric_data_)


pipeline = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector_num', get_numeric_data)
            ])),
             ('text_features', Pipeline([
                ('selector_text', get_text_data),
                ('vectorizer', TfidfVectorizer(max_df=0.5, max_features=2000, min_df=5)),
            ]))
         ])),
     ('lr', LogisticRegression(max_iter=1000, penalty='none'))
])


pipeline.fit(X, y)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('numeric_features',
                                                 Pipeline(steps=[('selector_num',
                                                                  FunctionTransformer(func=<function get_numeric_data_ at 0x7fe24a4070d0>))])),
                                                ('text_features',
                                                 Pipeline(steps=[('selector_text',
                                                                  FunctionTransformer(func=<function get_text_data_ at 0x7fe249c5a9d0>)),
                                                                 ('vectorizer',
                                                                  TfidfVectorizer(max_df=0.5,
                                                                                  max_features=2000,
                                                                                  min_df=5))]))])),
              

In [21]:
filename = '../classifiers/classifier_politician_ndtfr.sav'
joblib.dump(pipeline, filename)

['../classifiers/classifier_politician_ndtfr.sav']